In [1]:
import pandas as pd
import os, sys

data = dict()

# extract sun rise/set times from insane file

In [2]:

idx = 0
rawpath = os.path.join('..','raw','london-sun-rise-set.txt')
with open(rawpath, newline='') as infil:
    raw = infil.readlines()

source = [ line.strip() for line in raw ]
source = [ line for line in source if len(line)>0 ]
idx+=1 

# extract months
months = source[idx].split()

idx+=1 # skip month line
idx+=1 # skip rise/set line
idx+=1 # skip 'h m' line

running=True
while running:
    part = source[idx].split('   ')
    try:
        day = int(part.pop(0))
    except:
        break
    
    pidx = 0
    for month in months:
        
        if pidx >= len(part):
            break
            
        if len(part[pidx]) > 0:
            mno = months.index(month)+1
            data[(2013, mno, day)] =  \
                [ tuple(map(int, p.split(' '))) for p in part[pidx].split('  ') ]
            
            pidx+=1
        else:
            pidx+=5  
    idx+=1

# skip next header segment
idx+=9

# extract new months
new_months = source[idx].split()
months.extend(new_months)

idx+=1 # skip month line
idx+=1 # skip rise/set line
idx+=1 # skip 'h m' line

running=True
while running:
    part = source[idx].split('   ')
    
    try:
        day = int(part.pop(0))
    except:
        break
    
    pidx = 0
    for month in new_months:
        
        if pidx >= len(part):
            break
            
        if len(part[pidx]) > 0:
            mno = months.index(month)+1
            data[(2013, mno, day)] = \
                [ tuple(map(int, p.split(' '))) for p in part[pidx].split('  ') ]
            pidx+=1
        else:
            pidx+=5  
    idx+=1


# generate sane rise/set times

In [3]:
date = []
sunrise = []
sunset = []

for k in data:
    date.append("{}-{}-{}".format(*k))
    sunrise.append( "{}:{}:00".format(*(data[k][0])))
    sunset.append( "{}:{}:00".format(*(data[k][1])))
    

In [5]:
df = pd.DataFrame({
    'date': pd.to_datetime(date),
    'sunrise': pd.to_timedelta(sunrise),
    'sunset': pd.to_timedelta(sunset)
})
df.set_index('date', inplace=True)

In [6]:
df.to_csv('../data/london-sunrise-sunset.csv')